# Аналіз набору даних Students Social Media Addiction

### Блок 1: Первинний аналіз та аудит якості даних  

🎯 **Мета етапу**

Провести експрес-аудит структури датасету, оцінити повноту даних та визначити необхідні кроки з їх підготовки до аналізу.

📝 **Завдання**

- Завантаження вихідного набору даних 
- Аналіз розмірності та типів ознак  
- Перевірка цілісності даних (пошук дублікатів та пропущених значень)  
- Отримання базових статистичних показників


In [1]:
import pandas as pd
import numpy as np

# 1. Завантаження датасету
df = pd.read_csv('Students Social Media Addiction.csv')

# 2. Попередній перегляд перших 5 рядків
print("Перші 5 записів датасету:")
display(df.head())

# 3. Перевірка структури даних та типів колонок
print("\nІнформація про типи даних та пропуски:")
df.info()

# 4. Перевірка на наявність дублікатів
duplicates = df.duplicated().sum()
print(f"\nКількість дублікатів у даних: {duplicates}")

# 5. Перевірка на пропуски (null values)
missing_values = df.isnull().sum()
print("\nКількість пропущених значень у кожній колонці:")
print(missing_values)

# 6. Опис числових даних (мінімум, максимум, середнє)
print("\nСтатистичний опис числових колонок:")
display(df.describe())

Перші 5 записів датасету:


,Student_ID,Age,Gender,Academic_Level,Country,Avg_Daily_Usage_Hours,Most_Used_Platform,Affects_Academic_Performance,Sleep_Hours_Per_Night,Mental_Health_Score,Relationship_Status,Conflicts_Over_Social_Media,Addicted_Score
0,1,19,Female,Undergraduate,Bangladesh,5.2,Instagram,Yes,6.5,6,In Relationship,3,8
1,2,22,Male,Graduate,India,2.1,Twitter,No,7.5,8,Single,0,3
2,3,20,Female,Undergraduate,USA,6.0,TikTok,Yes,5.0,5,Complicated,4,9
3,4,18,Male,High School,UK,3.0,YouTube,No,7.0,7,Single,1,4
4,5,21,Male,Graduate,Canada,4.5,Facebook,Yes,6.0,6,In Relationship,2,7



Інформація про типи даних та пропуски:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Student_ID                    705 non-null    int64  
 1   Age                           705 non-null    int64  
 2   Gender                        705 non-null    object 
 3   Academic_Level                705 non-null    object 
 4   Country                       705 non-null    object 
 5   Avg_Daily_Usage_Hours         705 non-null    float64
 6   Most_Used_Platform            705 non-null    object 
 7   Affects_Academic_Performance  705 non-null    object 
 8   Sleep_Hours_Per_Night         705 non-null    float64
 9   Mental_Health_Score           705 non-null    int64  
 10  Relationship_Status           705 non-null    object 
 11  Conflicts_Over_Social_Media   705 non-null    int64  
 12  Addicted_Score          

,Student_ID,Age,Avg_Daily_Usage_Hours,Sleep_Hours_Per_Night,Mental_Health_Score,Conflicts_Over_Social_Media,Addicted_Score
count,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000
mean,353.000000,20.659574,4.918723,6.868936,6.226950,2.849645,6.436879
std,203.660256,1.399217,1.257395,1.126848,1.105055,0.957968,1.587165
min,1.000000,18.000000,1.500000,3.800000,4.000000,0.000000,2.000000
25%,177.000000,19.000000,4.100000,6.000000,5.000000,2.000000,5.000000
50%,353.000000,21.000000,4.800000,6.900000,6.000000,3.000000,7.000000
75%,529.000000,22.000000,5.800000,7.700000,7.000000,4.000000,8.000000
max,705.000000,24.000000,8.500000,9.600000,9.000000,5.000000,9.000000


### Висновки до Блоку 1

#### Якість даних:  
  Висока. Датасет є повним, пропуски та дублікати відсутні.

#### Обсяг вибірки:  
  **705** записів є достатньою базою для отримання статистично значущих висновків.

#### Необхідність трансформації:  
  Виявлено, що колонка **Affects_Academic_Performance** має текстовий формат (*Yes/No*).  
  👉 Для проведення кореляційного аналізу її необхідно конвертувати у бінарний числовий формат (**1/0**).


### Блок 2: Очищення та трансформація даних  

#### 🎯 Мета етапу

Підготовка даних до математичного моделювання та візуалізації шляхом створення нових ознак (*Feature Engineering*) та приведення типів даних до потрібного формату.

#### 📝 Завдання

- Створення резервної робочої копії датасету
- Конвертація впливу на навчання у числовий формат  
  (колонка *Affects_Academic_Performance* → тип *int*, значення *Yes/No* → **1/0**)  
- Створення нової колонки Addiction_Level  
  (рівень залежності на основі *Addicted_Score* для кластеризації та групування)  
- Перевірка структури оновленого набору даних через `.info()`  
- Збереження очищеного датасету у новий файл  
  (формат **CSV** для стабільної роботи в *Streamlit*)  

#### 📌 Для чого
Щоб підготувати дані до візуалізації та побудови моделей машинного навчання.


In [2]:
# 1. Створюємо копію датасету, щоб оригінал залишався незмінним
df_cleaned = df.copy()

# 2. Трансформація Affects_Academic_Performance у числовий формат
# 'Yes' перетворюємо на 1, 'No' — на 0
df_cleaned['Affects_Academic_Performance_Numeric'] = df_cleaned['Affects_Academic_Performance'].map({'Yes': 1, 'No': 0})

# 3. Створення категоріальної змінної Addiction_Level
# Розділимо оцінку від 1 до 10 на три рівні:
# 1-3: Low (Низький), 4-7: Medium (Середній), 8-10: High (Високий)
def categorize_addiction(score):
    if score <= 3:
        return 'Low'
    elif score <= 7:
        return 'Medium'
    else:
        return 'High'

# Застосовуємо функцію до колонки Addicted_Score
df_cleaned['Addiction_Level'] = df_cleaned['Addicted_Score'].apply(categorize_addiction)

# 4. Перевірка структури оновленого датасету
print("Інформація про очищений датасет:")
df_cleaned.info()

# 5. Додатково глянемо на перші кілька рядків, щоб побачити нові колонки в кінці
print("\nПерші 5 рядків очищених даних:")
display(df_cleaned.head())

# 6. Збереження очищеного датасету у новий CSV файл для подальшої роботи
df_cleaned.to_csv('cleaned_social_media_data.csv', index=False)
print("\nФайл 'cleaned_social_media_data.csv' успішно створено та збережено!")

Інформація про очищений датасет:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Student_ID                            705 non-null    int64  
 1   Age                                   705 non-null    int64  
 2   Gender                                705 non-null    object 
 3   Academic_Level                        705 non-null    object 
 4   Country                               705 non-null    object 
 5   Avg_Daily_Usage_Hours                 705 non-null    float64
 6   Most_Used_Platform                    705 non-null    object 
 7   Affects_Academic_Performance          705 non-null    object 
 8   Sleep_Hours_Per_Night                 705 non-null    float64
 9   Mental_Health_Score                   705 non-null    int64  
 10  Relationship_Status                   705 non-null   

,Student_ID,Age,Gender,Academic_Level,Country,Avg_Daily_Usage_Hours,Most_Used_Platform,Affects_Academic_Performance,Sleep_Hours_Per_Night,Mental_Health_Score,Relationship_Status,Conflicts_Over_Social_Media,Addicted_Score,Affects_Academic_Performance_Numeric,Addiction_Level
0,1,19,Female,Undergraduate,Bangladesh,5.2,Instagram,Yes,6.5,6,In Relationship,3,8,1,High
1,2,22,Male,Graduate,India,2.1,Twitter,No,7.5,8,Single,0,3,0,Low
2,3,20,Female,Undergraduate,USA,6.0,TikTok,Yes,5.0,5,Complicated,4,9,1,High
3,4,18,Male,High School,UK,3.0,YouTube,No,7.0,7,Single,1,4,0,Medium
4,5,21,Male,Graduate,Canada,4.5,Facebook,Yes,6.0,6,In Relationship,2,7,1,Medium



Файл 'cleaned_social_media_data.csv' успішно створено та збережено!


### Висновки до Блоку 2:
- Готовність до аналізу: Affects_Academic_Performance_Numeric (тип int64) дозволяє математично виміряти силу зв'язку між соцмережами та успішністю.

- Сегментація: Нова колонка Addiction_Level (тип object) спрощує групування студентів для порівняльних графіків (наприклад, аналіз тривалості сну за групами ризику).

- Кількість колонок: тепер їх 15 (було 13 + 2 нові).

### Блок 3: Кореляційний аналіз та перевірка гіпотез (Гіпотези 1 та 2)
Що ми робимо:
- Будуємо матрицю кореляції для всіх числових показників.
- Обчислюємо конкретні коефіцієнти для Гіпотези 1 (Зв'язок використання та сну) та Гіпотези 2 (Зв'язок залежності та психічного здоров'я).
- Групуємо дані за Addiction_Level, щоб побачити різницю в "якості життя" між групами.

Для чого: Щоб мовою цифр підтвердити наші припущення. Наприклад, якщо кореляція між використанням та сном буде від'ємною (наприклад, -0.6), ми зможемо математично стверджувати: "Зі зростанням часу в соцмережах якість сну суттєво падає".

In [3]:
# 1. Вибираємо числові колонки для аналізу кореляцій
# Включаємо Age, Usage, Sleep, Mental Health, Conflicts, Addiction Score та нашу нову числову успішність
numerical_cols = [
    'Age', 'Avg_Daily_Usage_Hours', 'Sleep_Hours_Per_Night', 
    'Mental_Health_Score', 'Conflicts_Over_Social_Media', 
    'Addicted_Score', 'Affects_Academic_Performance_Numeric'
]

# Створюємо матрицю кореляції
corr_matrix = df_cleaned[numerical_cols].corr()

print("Матриця кореляції (вплив факторів один на одного):")
display(corr_matrix)

# 2. Перевірка Гіпотези 1: Чи зменшується сон при збільшенні використання?
hypo1_corr = df_cleaned['Avg_Daily_Usage_Hours'].corr(df_cleaned['Sleep_Hours_Per_Night'])
print(f"\nКореляція (Використання vs Сон): {hypo1_corr:.2f}")

# 3. Перевірка Гіпотези 2: Чи страждає психічне здоров'я при зростанні залежності?
hypo2_corr = df_cleaned['Addicted_Score'].corr(df_cleaned['Mental_Health_Score'])
print(f"Кореляція (Залежність vs Психічне здоров'я): {hypo2_corr:.2f}")

# 4. Порівняння груп за рівнем залежності (Addiction_Level)
# Подивимось на середній сон та психічне здоров'я для категорій Low, Medium, High
group_comparison = df_cleaned.groupby('Addiction_Level')[['Sleep_Hours_Per_Night', 'Mental_Health_Score', 'Avg_Daily_Usage_Hours']].mean()

print("\nПорівняння середніх показників за рівнями залежності:")
display(group_comparison)

Матриця кореляції (вплив факторів один на одного):


,Age,Avg_Daily_Usage_Hours,Sleep_Hours_Per_Night,Mental_Health_Score,Conflicts_Over_Social_Media,Addicted_Score,Affects_Academic_Performance_Numeric
Age,1.000000,-0.113682,0.125265,0.160278,-0.184482,-0.166396,-0.137140
Avg_Daily_Usage_Hours,-0.113682,1.000000,-0.790582,-0.801058,0.804582,0.832000,0.661474
Sleep_Hours_Per_Night,0.125265,-0.790582,1.000000,0.707439,-0.677266,-0.764858,-0.625373
Mental_Health_Score,0.160278,-0.801058,0.707439,1.000000,-0.893572,-0.945051,-0.808921
Conflicts_Over_Social_Media,-0.184482,0.804582,-0.677266,-0.893572,1.000000,0.933586,0.832030
Addicted_Score,-0.166396,0.832000,-0.764858,-0.945051,0.933586,1.000000,0.866049
Affects_Academic_Performance_Numeric,-0.137140,0.661474,-0.625373,-0.808921,0.832030,0.866049,1.000000



Кореляція (Використання vs Сон): -0.79
Кореляція (Залежність vs Психічне здоров'я): -0.95

Порівняння середніх показників за рівнями залежності:


,Sleep_Hours_Per_Night,Mental_Health_Score,Avg_Daily_Usage_Hours
Addiction_Level,,,
High,5.728643,4.974874,6.256784
Low,8.135294,8.058824,2.888235
Medium,7.288957,6.672802,4.444785


Знак кореляції: Мінус (-) означає, що коли один показник росте, інший падає.

### Висновки до Блоку 3:**
- _Гіпотеза 1 підтверджена_: Кореляція -0.79 між часом використання та сном свідчить про сильний негативний зв'язок. Студенти, які проводять більше часу в соцмережах, жертвують сном. У групи з високою залежністю сон становить у середньому лише 5.7 годин, що значно нижче норми.
- _Гіпотеза 2 підтверджена_ (екстремальний зв'язок): Кореляція -0.95 між залежністю та психічним здоров'ям є майже лінійною. Це означає, що рівень залежності є ключовим предиктором (показником) психологічного стану студента в цьому наборі даних.
- _Додатковий інсайт:_ Також спостерігається дуже висока кореляція залежності з конфліктами у стосунках (0.93) та впливом на навчання (0.87). Це свідчить про те, що проблема є комплексною і зачіпає всі сфери життя.

### Блок 4: Аналіз платформ, соціальних факторів та успішності (Гіпотези 3, 4, 5 та 6)
Що ми робимо:
- Аналіз платформ (Гіпотеза 3): Розраховуємо середній рівень залежності для кожної платформи.
- Конфлікти у стосунках (Гіпотеза 4): Порівнюємо середню кількість конфліктів у стосунках через соц.мережі для різних сімейних статусів, щоб перевірити, чи є статус "Complicated" найбільш проблемним.
- Вплив на навчання (Гіпотеза 5): Використовуємо нашу категорію Addiction_Level == 'High', щоб визначити відсоток студентів, які відчувають негативний вплив на навчання.
- Стосунки як захист (Гіпотеза 6): Аналізуємо середній бал залежності (Addicted_Score) для кожного статусу стосунків, щоб побачити, чи дійсно стабільні стосунки ("In Relationship") знижують ризик залежності.

**Для чого:** Щоб виявити конкретні "групи ризику" та зовнішні чинники (платформи, сімейний стан), які сприяють або, навпаки, запобігають розвитку залежності.

In [4]:
# 1. Аналіз платформ (Гіпотеза 3)
# Групуємо за платформою та рахуємо середній бал залежності
platform_addiction = df_cleaned.groupby('Most_Used_Platform')['Addicted_Score'].mean().sort_values(ascending=False)

print("Середній показник залежності за платформами (Гіпотеза 3):")
print(platform_addiction)

# 2. Аналіз конфліктів через соцмережі залежно від статусу (Гіпотеза 4)
# Перевіряємо, де найбільше конфліктів через соцмережі
relationship_conflicts = df_cleaned.groupby('Relationship_Status')['Conflicts_Over_Social_Media'].mean().sort_values(ascending=False)

print("\nСередня кількість конфліктів через соцмережі за статусом стосунків (Гіпотеза 4):")
print(relationship_conflicts)

# 3. Стосунки як захисний фактор (Гіпотеза 5)
# Порівнюємо рівень залежності між різними статусами стосунків
relationship_protection = df_cleaned.groupby('Relationship_Status')['Addicted_Score'].mean().sort_values()

print("\nСередній бал залежності за статусом стосунків (Гіпотеза 5):")
print(relationship_protection)

# 4. Вплив високої залежності на навчання (Гіпотеза 6)
# Використовуємо створену нами категорію 'High' з Addiction_Level
high_addiction_group = df_cleaned[df_cleaned['Addiction_Level'] == 'High']
academic_impact = high_addiction_group['Affects_Academic_Performance'].value_counts(normalize=True) * 100

print("\nВплив на навчання серед студентів з високим рівнем залежності (Гіпотеза 6):")
if 'Yes' in academic_impact:
    print(f"Відсоток студентів, що відчули вплив: {academic_impact['Yes']:.1f}%")
else:
    print("Дані про негативний вплив відсутні")

Середній показник залежності за платформами (Гіпотеза 3):
Most_Used_Platform
WhatsApp     7.462963
Snapchat     7.461538
TikTok       7.428571
Instagram    6.554217
YouTube      6.100000
WeChat       6.066667
KakaoTalk    6.000000
Facebook     5.666667
Twitter      5.500000
VKontakte    5.000000
LinkedIn     3.809524
LINE         3.000000
Name: Addicted_Score, dtype: float64

Середня кількість конфліктів через соцмережі за статусом стосунків (Гіпотеза 4):
Relationship_Status
Complicated        3.031250
Single             2.901042
In Relationship    2.761246
Name: Conflicts_Over_Social_Media, dtype: float64

Середній бал залежності за статусом стосунків (Гіпотеза 5):
Relationship_Status
In Relationship    6.342561
Single             6.458333
Complicated        7.031250
Name: Addicted_Score, dtype: float64

Вплив на навчання серед студентів з високим рівнем залежності (Гіпотеза 6):
Відсоток студентів, що відчули вплив: 100.0%


### Висновки до Блоку 4:
- _Гіпотеза 3 (Платформи):_ Підтверджена частково. WhatsApp, Snapchat та TikTok очолюють рейтинг (більше $7.4$). Цікаво, що месенджери з елементами соцмереж (WhatsApp, Snapchat) виявилися навіть вищими за Instagram. LinkedIn очікувано має найнижчий рівень залежності ($3.8$), що підтверджує його професійну спрямованість. 
- _Гіпотеза 4 (Конфлікти):_ Підтверджена. Студенти зі статусом "Complicated" мають найбільше конфліктів ($3.03$), тоді як ті, хто "In Relationship", конфліктують найменше ($2.76$).
- _Гіпотеза 5 (Захисний фактор):_ Підтверджена. Студенти у стабільних стосунках мають найнижчий середній бал залежності ($6.34$), тоді як у групі "Complicated" він найвищий ($7.03$). Це доводить, що емоційна стабільність у реальному житті знижує тягу до віртуального.
- _Гіпотеза 6 (Навчання):_ Підтверджена на $100\%$. Кожен студент у з рівнем залежності "High" відмітив негативний вплив на академічну успішність. Це критичний інсайт.

### Блок 5: Підготовка до Машинного навчання (Кластеризація)

Тепер спробуємо згрупувати студентів не за однією ознакою, а за сукупністю факторів (час, сон, здоров'я, залежність).

#### Що ми робимо:
- Масштабування (Scaling) за допомогою StandardScaler для усунення впливу різниці в одиницях вимірювання. Оскільки різні показники мають різні одиниці (години сну — від 4 до 9, а бал залежності — від 1 до 10), їх треба привести до одного масштабу.
- K-Means Clustering: Математичний поділ вибірки на 3 сегменти (кластери).
- Інтерпретація: Подивимось, що це за групи (наприклад: "Свідомі користувачі", "Група ризику", "Залежні").
- Оновлення даних: Збереження міток кластерів у фінальний датасет для інтерактивної діагностики.

#### Для чого:
Це дозволить нам не просто аналізувати окремі цифри, а створити "портрети" типових студентів.

In [5]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1. Вибираємо ключові метрики для кластеризації
features_for_clustering = [
    'Avg_Daily_Usage_Hours', 
    'Sleep_Hours_Per_Night', 
    'Mental_Health_Score', 
    'Addicted_Score'
]

x = df_cleaned[features_for_clustering]

# 2. Стандартизація даних (щоб усі ознаки мали однаковий вплив)
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# 3. Використовуємо метод K-Means для створення 3 кластерів
# (Кількість кластерів 3 відповідає нашим рівням Low, Medium, High, але тепер на основі всіх даних)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_cleaned['Cluster'] = kmeans.fit_predict(x_scaled)

# 4. Аналіз отриманих кластерів
cluster_summary = df_cleaned.groupby('Cluster')[features_for_clustering].mean()

print("Середні показники для кожного кластера (портрети студентів):")
display(cluster_summary)


# 5. Оновлюємо наш файл, щоб зберегти результати кластеризації
df_cleaned.to_csv('cleaned_social_media_data.csv', index=False)
print("\nДані з кластерами збережено в 'cleaned_social_media_data.csv'")

Середні показники для кожного кластера (портрети студентів):


,Avg_Daily_Usage_Hours,Sleep_Hours_Per_Night,Mental_Health_Score,Addicted_Score
Cluster,,,,
0,3.839858,7.820285,7.380783,4.740214
1,6.475824,5.581868,4.978022,8.241758
2,5.000413,6.732231,5.826446,7.049587



Дані з кластерами збережено в 'cleaned_social_media_data.csv'


### Висновки до Блоку 5:

Тепер ми маємо не просто цифри, а чіткі профілі студентів. Машинне навчання допомогло нам виділити три різні стратегії поведінки.

Інтерпретація результатів кластеризації:
На основі отриманих середніх значень, ми можемо дати назви цим групам (кластерам) для Storytelling:

#### 1. Кластер 0 — "Збалансовані" (Balanced Users):

Характеристики: Найнижче використання (~3.8 год), найкращий сон (~7.8 год) та найвищий рівень психічного здоров'я (~7.4).

Висновок: Це студенти, які використовують соцмережі як інструмент, а не як спосіб життя.

#### 2. Кластер 2 — "Група ризику" (At-Risk Group):

Характеристики: Помірне використання (~5 год), сон вже нижче норми (~6.7 год), високий середній бал залежності (~7.0).

Висновок: Ця група вже відчуває негативні наслідки, але ще зберігає відносну стабільність. Саме на них мають бути спрямовані профілактичні заходи.

#### 3. Кластер 1 — "Цифрова залежність" (Highly Addicted):

Характеристики: Критично високе використання (~6.5 год), хронічний недосип (~5.6 год) та найнижчі показники психічного здоров'я (~5.0). Найвищий бал залежності (~8.2).

Висновок: Ця група студентів потребує серйозної цифрової детоксикації та, можливо, психологічної допомоги.

_Результати цієї кластеризації закладені в основу модуля ML-діагностики дашборду в Streamlit._

### Блок 6: Агрегація даних за географічним принципом (Регіоналізація)

#### Мета етапу:
Оскільки вихідний набір даних містить велику кількість країн з одиничними відповідями, прямий порівняльний аналіз по країнах є нерепрезентативним. Для отримання статистично значущих результатів ми впроваджуємо групування країн у макрорегіони (континенти).

#### Завдання:

1. Геокодування: Автоматичне присвоєння кожній країні відповідного континенту за допомогою бібліотеки pycountry_convert з використанням стандартів ISO.
2. Обробка винятків: Створення словника коригування для назв країн, що не відповідають формату ISO (наприклад, США, Великобританія, Ватикан).
3. Збагачення датасету: Створення нової аналітичної ознаки Region, яка стане основою для подальшої візуалізації на карті світу.

In [10]:
# 1. Встановлення бібліотеки (це потрібно зробити один раз)
!pip install pycountry_convert

import pandas as pd
from pycountry_convert import country_name_to_country_alpha2, country_alpha2_to_continent_code

def get_continent(country_name):
    # Пряме виправлення для Ватикану та інших специфічних випадків
    if country_name == 'Vatican City':
        return 'Europe'
    
    try:
        manual_fixes = {
            'USA': 'US', 'UK': 'GB', 'South Korea': 'KR', 'UAE': 'AE', 
            'Russia': 'RU', 'Vietnam': 'VN', 'Czech Republic': 'CZ', 
            'Trinidad': 'TT', 'Kosovo': 'RS', 'Bosnia': 'BA',
            'Bahamas': 'BS', 'Hong Kong': 'HK', 'Taiwan': 'TW'
        }
        
        country_code = manual_fixes.get(country_name)
        if not country_code:
            country_code = country_name_to_country_alpha2(country_name)
            
        continent_code = country_alpha2_to_continent_code(country_code)
        
        continent_map = {
            'AF': 'Africa', 'AS': 'Asia', 'EU': 'Europe', 
            'NA': 'North America', 'SA': 'South America', 'OC': 'Oceania'
        }
        return continent_map.get(continent_code, "Other")
    except:
        return "Other"

# 1. Застосовуємо фінальну функцію
df_cleaned['Region'] = df_cleaned['Country'].apply(get_continent)

# 2. Перевіряємо, чи зникла категорія "Other"
print("Фінальний розподіл за регіонами (100% покриття):")
print(df_cleaned['Region'].value_counts())

# 3. Перевірка Гіпотези 7 з усіма даними
final_region_stats = df_cleaned.groupby('Region')['Addicted_Score'].mean().sort_values(ascending=False)
print("\nРівень залежності за регіонами:")
print(final_region_stats)

# 4. Остаточне збереження очищеного файлу
df_cleaned.to_csv('cleaned_social_media_data.csv', index=False)
print("\nФайл 'cleaned_social_media_data.csv' повністю готовий та збережений!")

# 4. Перевірка структури оновленого датасету
print("Інформація про очищений датасет:")
df_cleaned.info()


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Фінальний розподіл за регіонами (100% покриття):
Region
Europe           278
Asia             276
North America    106
Oceania           22
South America     17
Africa             6
Name: count, dtype: int64

Рівень залежності за регіонами:
Region
North America    7.575472
South America    6.764706
Asia             6.673913
Africa           6.166667
Europe           5.827338
Oceania          5.500000
Name: Addicted_Score, dtype: float64

Файл 'cleaned_social_media_data.csv' повністю готовий та збережений!
Інформація про очищений датасет:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Student_ID                            705 non-null    int64  
 1   Age                                 

### Висновок до Блоку 6:

- Успішність трансформації: Усі 705 записів успішно класифіковані за регіональною ознакою. Це усунуло проблему "малих чисел" і підготувало дані для побудови глобальних дашбордів.
- Перевірка Регіональної гіпотези (№7): Попередній розрахунок підтвердив нерівномірність цифрової залежності. Північна Америка демонструє найвищий рівень залежності ($7.58$), тоді як Океанія та Європа показують помірніші результати ($5.50$ та $5.83$ відповідно).
- Готовність даних: Додавання ознаки Region дозволяє перейти від простої статистики до геопросторового аналізу та сегментації ринку споживання контенту.

### Блок 7: Функціональна сегментація платформ  
*(Класифікація за типом контенту)*

#### 🎯 Мета етапу
Вийти за межі аналізу окремих платформ (Instagram, TikTok тощо) та дослідити вплив конкретних механік споживання контенту на рівень залежності.  
Це дозволить виявити, які саме типи цифрового середовища є найбільш адиктивними.

#### 📝 Завдання

- **Категоризація:**  
  Групування платформ у чотири класи:  
  - *Entertain-Scroll* (алгоритмічний контент)  
  - *Social-Network* (інформаційні стрічки)  
  - *Direct-Messaging* (комунікація)  
  - *Professional* (кар'єра)

- **Сегментація:**  
  Створення нової аналітичної ознаки **Platform_Type** для збагачення датасету.

- **Гендерно-функціональний аналіз:**  
  Визначення відмінностей у виборі контенту між чоловічою та жіночою аудиторією та розрахунок середніх показників залежності для кожної групи.


In [7]:
# 1. Визначаємо мапінг категорій
platform_map = {
    'TikTok': 'Entertain-Scroll', 'Instagram': 'Entertain-Scroll', 
    'YouTube': 'Entertain-Scroll', 'Snapchat': 'Entertain-Scroll',
    'Facebook': 'Social-Network', 'Twitter': 'Social-Network', 'VKontakte': 'Social-Network',
    'WhatsApp': 'Direct-Messaging', 'LINE': 'Direct-Messaging', 
    'WeChat': 'Direct-Messaging', 'KakaoTalk': 'Direct-Messaging',
    'LinkedIn': 'Professional'
}

# 2. Додаємо колонку категорії в основний DataFrame
df['Platform_Type'] = df['Most_Used_Platform'].map(platform_map)

# 3. Створюємо таблицю гендерного розподілу
gender_type_analysis = df.groupby(['Platform_Type', 'Gender']).size().unstack(fill_value=0)

# 4. Додаємо колонку "Всього" (сума хлопців та дівчат)
gender_type_analysis['Всього'] = gender_type_analysis['Female'] + gender_type_analysis['Male']

# 5. Розраховуємо середні показники залежності для кожної групи
type_metrics = df.groupby('Platform_Type').agg({
    'Addicted_Score': 'mean',
    'Avg_Daily_Usage_Hours': 'mean'
}).round(2)

# 6. Об'єднуємо все в одну підсумкову таблицю
final_report = gender_type_analysis.merge(type_metrics, left_index=True, right_index=True)

# Перейменовуємо для зрозумілості
final_report = final_report.rename(columns={
    'Female': 'Дівчата',
    'Male': 'Хлопці',
    'Addicted_Score': 'Сер. Залежність',
    'Avg_Daily_Usage_Hours': 'Сер. Час (год)'
})

print("📊 Глибокий аналіз за категоріями платформ:")
print(final_report.sort_values('Всього', ascending=False))

# 7. Зберігаємо оновлений датасет з новою колонкою Platform_Type
df.to_csv('cleaned_social_media_data.csv', index=False)
print("\nФайл 'cleaned_social_media_data.csv' успішно оновлено: додано колонку Platform_Type.")

📊 Глибокий аналіз за категоріями платформ:
                  Дівчата  Хлопці  Всього  Сер. Залежність  Сер. Час (год)
Platform_Type                                                             
Entertain-Scroll      266     160     426             6.89            5.03
Social-Network         40     125     165             5.59            4.55
Direct-Messaging       39      54      93             6.47            5.59
Professional            8      13      21             3.81            2.52

Файл 'cleaned_social_media_data.csv' успішно оновлено: додано колонку Platform_Type.


### Висновок до Блоку 7:
Тепер у нас є доказова база для обґрунтування психологічної природи залежності.

#### 🧠 Ключові висновки

- **"Дофамінова пастка" (Entertain-Scroll):**  
  Найпопулярніша категорія (426 студентів) має найвищий рівень залежності **6.89**, попри те, що час використання не є найбільшим.  
  👉 Це доводить, що алгоритми коротких відео викликають швидшу та глибшу адикцію, ніж просте спілкування.

- **Парадокс месенджерів (Direct-Messaging):**  
  Найвищий середній час використання — **5.59 год**, але рівень залежності **6.47** нижчий, ніж у розважальних мережах.  
  👉 Висновок: студенти проводять багато часу в месенджерах, бо це інструмент для життя (навчання, координація), але це рідше стає хворобливою залежністю порівняно з TikTok/Instagram.

- **Гендерний розрив у Social-Network:**  
  Хлопці використовують "новинні" мережі (Facebook, Twitter) у **3 рази частіше** за дівчат (125 проти 40).  
  👉 Це вказує на різницю в цілях: хлопці йдуть за інформацією, дівчата — за візуальним контентом.

- **Professional (LinkedIn) як "безпечна зона":**  
  Найнижча залежність — **3.81** та мінімальний час.  
  👉 Це підтверджує, що професійні мережі не мають адиктивних механік та адиктивного потенціалу для молоді.


### Блок 8: Геопросторова диференціація платформ 

### 🎯 Мета етапу
Визначити географічні центри тяжіння для різних соціальних медіа та виявити регіональні особливості споживання цифрового контенту.

### 📝 Завдання

- **Регіональний топ:**  
  Визначення найбільш затребуваної платформи для кожного макрорегіону.

- **Аналіз ядра аудиторії:**  
  Ідентифікація регіону, який забезпечує найбільшу кількість користувачів для кожної окремої мережі.
- **Підготовка до мапування:** Формування зведених даних для побудови інтерактивної "Карти цифрових лідерів".

In [11]:
import pandas as pd

# 1. Завантажуємо дані
df = pd.read_csv('cleaned_social_media_data.csv')

# --- ВАРІАНТ А: Найпопулярніша платформа в кожному регіоні ---
# Групуємо за регіоном та платформою, рахуємо кількість користувачів
region_platform_counts = df.groupby(['Region', 'Most_Used_Platform']).size().reset_index(name='User_Count')

# Знаходимо платформу з максимальним значенням для кожного регіону
top_platform_per_region = region_platform_counts.loc[region_platform_counts.groupby('Region')['User_Count'].idxmax()]

print("📍 ТОП-платформа для кожного регіону:")
print(top_platform_per_region.sort_values('User_Count', ascending=False).to_string(index=False))

print("\n" + "="*50 + "\n")

# --- ВАРІАНТ Б: Де платформа має найбільше прихильників? ---

# 1. Рахуємо кількість користувачів для кожної пари Платформа-Регіон
platform_region_counts = df.groupby(['Most_Used_Platform', 'Region']).size().reset_index(name='Кількість')

# 2. Для кожної платформи знаходимо рядок з максимальною кількістю користувачів
# idxmax() знайде індекс рядка, де 'Кількість' найбільша для кожної групи Most_Used_Platform
top_region_per_platform = platform_region_counts.loc[platform_region_counts.groupby('Most_Used_Platform')['Кількість'].idxmax()]

# 3. Перевпорядковуємо колонки (Платформа спочатку) та сортуємо за популярністю
top_region_per_platform = top_region_per_platform[['Most_Used_Platform', 'Region', 'Кількість']]
top_region_per_platform = top_region_per_platform.rename(columns={
    'Most_Used_Platform': 'Соціальна мережа',
    'Region': 'Основний регіон (ядро)'
})

print("🌍 Де кожна платформа найпопулярніша:")
print(top_region_per_platform.sort_values('Кількість', ascending=False).to_string(index=False))

📍 ТОП-платформа для кожного регіону:
       Region Most_Used_Platform  User_Count
       Europe          Instagram         119
         Asia          Instagram          76
North America          Instagram          38
      Oceania          Instagram          12
South America             TikTok           5
       Africa           Facebook           1


🌍 Де кожна платформа найпопулярніша:
Соціальна мережа Основний регіон (ядро)  Кількість
       Instagram                 Europe        119
          TikTok                   Asia         66
        Facebook                 Europe         64
        WhatsApp                   Asia         32
         Twitter                 Europe         16
          WeChat                   Asia         15
            LINE                   Asia         12
       KakaoTalk                   Asia         12
       VKontakte                 Europe         12
        LinkedIn                 Europe         10
        Snapchat                 Europe         

### Висновок до Блоку 8:

- **Гегемонія Instagram:**  
  Лідер майже на всіх континентах (Європа, Азія, Пн. Америка, Океанія).  
  👉 Це підтверджує, що візуальний контент — глобальний тренд №1 серед студентів.

- **Регіональна специфіка:**  
  TikTok захопив лідерство в **Південній Америці**.

- **Facebook в Африці:**  
  Залишається головним, навіть якщо там лише 1 респондент — показово для вибірки.

- **Азійський "ексклюзив":**  
  WeChat, LINE, KakaoTalk мають своє ядро саме в **Азії**.  
  👉 Це відображає унікальність локальних екосистем.

- **Європа як "хаб" багатьох мереж:**  
  Регіон став основним центром активності для найбільшої кількості мереж (Facebook, Twitter, LinkedIn, Snapchat, VK), що свідчить про високу диверсифікацію цифрового споживання.


## 🎉 Підсумок Етапу 1: Дослідницький аналіз та підготовка даних (DataLab)

У межах першого етапу було успішно реалізовано наступні кроки:

---

### 🧹 Data Cleaning & Quality Control  
*(Очищення та контроль якості даних)*

- Проведено комплексний аудит повноти даних  
- Підтверджено відсутність пропущених значень та дублікатів  
  - **0 пропусків**
  - **0 дублікатів**
- Валідовано типи змінних та перевірено відповідність значень логічним діапазонам (наприклад, години сну не можуть бути від'ємними) 

---

### 🧠 Feature Engineering  
*(Трансформація та створення ознак)*

- Текстові показники (наприклад, `Affects_Academic_Performance`) конвертовано у числовий формат (1/0) для проведення математичного аналізу та кореляційних розрахунків
- Впроваджено нову категоріальну метрику **`Addiction_Level`**:
  - `Low`
  - `Medium`
  - `High`  
  для якісної оцінки стану респондентів
- Функціональна сегментація: Створено ознаку Platform_Type, що групує платформи за типом контенту (Entertain-Scroll, Messaging тощо)

---

### 📊 Статистична перевірка гіпотез

- Проведено статистичну перевірку **7 гіпотез**
- Кількісно доведено вплив соціальних мереж на ключові сфери життя студентів
- Виявлено **критично високі рівні кореляції** між цифровою залежністю та:
  - порушенням сну
  - погіршенням психічного стану
  - зниженням академічної успішності

---

### 🤖 Сегментація користувачів  
*(Machine Learning)*

- Виконано стандартизацію ознак за допомогою **StandardScaler**
- За допомогою алгоритму кластеризації **K-Means** автоматично виділено  
  **три стійкі профілі студентів**:
  - 🟢 **Balanced Users (Збалансовані)**
  - 🟡 **At-Risk Group (Група ризику)**
  - 🔴 **Highly Addicted (Цифрова залежність)**

---

### 🌍 Геопросторова агрегація  
*(Регіоналізація)*

- Реалізовано групування країн за континентами (регіонами)
- Використано бібліотеку **`pycountry_convert`** з ручним коригуванням винятків
- Це дозволило:
  - усунути *статистичний шум* від країн з поодинокими відповідями
  - підготувати підґрунтя для **макроаналізу**

---

### 💾 Збереження результатів

- Сформовано фінальний збагачений датасет  
  **`cleaned_social_media_data.csv`**
- Датасет виступає **"золотим стандартом"** для:
  - подальшої візуалізації
  - аналітики
  - розробки інтерактивного додатку

---

### ✅ Результат етапу

Дані пройшли **повний цикл обробки**  
*(від Raw Data до Clean Data)*  
та повністю готові до:

- інтеграції в **GitHub**
- деплою у **Streamlit**


In [23]:
# 1. Завантаження датасету
df = pd.read_csv('cleaned_social_media_data.csv')

# 2. Попередній перегляд перших 5 рядків
print("Перші 5 записів датасету:")
display(df.head())

# 3. Перевірка структури даних та типів колонок
print("\nІнформація про типи даних та пропуски:")
df.info()

# 4. Перевірка на наявність дублікатів
duplicates = df.duplicated().sum()
print(f"\nКількість дублікатів у даних: {duplicates}")

# 5. Перевірка на пропуски (null values)
missing_values = df.isnull().sum()
print("\nКількість пропущених значень у кожній колонці:")
print(missing_values)

# 6. Опис числових даних (мінімум, максимум, середнє)
print("\nСтатистичний опис числових колонок:")
display(df.describe())

Перші 5 записів датасету:


,Student_ID,Age,Gender,Academic_Level,Country,Avg_Daily_Usage_Hours,Most_Used_Platform,Affects_Academic_Performance,Sleep_Hours_Per_Night,Mental_Health_Score,Relationship_Status,Conflicts_Over_Social_Media,Addicted_Score,Affects_Academic_Performance_Numeric,Addiction_Level,Cluster,Region,Platform_Type
0,1,19,Female,Undergraduate,Bangladesh,5.2,Instagram,Yes,6.5,6,In Relationship,3,8,1,High,2,Asia,Entertain-Scroll
1,2,22,Male,Graduate,India,2.1,Twitter,No,7.5,8,Single,0,3,0,Low,0,Asia,Social-Network
2,3,20,Female,Undergraduate,USA,6.0,TikTok,Yes,5.0,5,Complicated,4,9,1,High,1,North America,Entertain-Scroll
3,4,18,Male,High School,UK,3.0,YouTube,No,7.0,7,Single,1,4,0,Medium,0,Europe,Entertain-Scroll
4,5,21,Male,Graduate,Canada,4.5,Facebook,Yes,6.0,6,In Relationship,2,7,1,Medium,2,North America,Social-Network



Інформація про типи даних та пропуски:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Student_ID                            705 non-null    int64  
 1   Age                                   705 non-null    int64  
 2   Gender                                705 non-null    object 
 3   Academic_Level                        705 non-null    object 
 4   Country                               705 non-null    object 
 5   Avg_Daily_Usage_Hours                 705 non-null    float64
 6   Most_Used_Platform                    705 non-null    object 
 7   Affects_Academic_Performance          705 non-null    object 
 8   Sleep_Hours_Per_Night                 705 non-null    float64
 9   Mental_Health_Score                   705 non-null    int64  
 10  Relationship_Status                   705 non-

,Student_ID,Age,Avg_Daily_Usage_Hours,Sleep_Hours_Per_Night,Mental_Health_Score,Conflicts_Over_Social_Media,Addicted_Score,Affects_Academic_Performance_Numeric,Cluster
count,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000
mean,353.000000,20.659574,4.918723,6.868936,6.226950,2.849645,6.436879,0.642553,0.944681
std,203.660256,1.399217,1.257395,1.126848,1.105055,0.957968,1.587165,0.479588,0.860136
min,1.000000,18.000000,1.500000,3.800000,4.000000,0.000000,2.000000,0.000000,0.000000
25%,177.000000,19.000000,4.100000,6.000000,5.000000,2.000000,5.000000,0.000000,0.000000
50%,353.000000,21.000000,4.800000,6.900000,6.000000,3.000000,7.000000,1.000000,1.000000
75%,529.000000,22.000000,5.800000,7.700000,7.000000,4.000000,8.000000,1.000000,2.000000
max,705.000000,24.000000,8.500000,9.600000,9.000000,5.000000,9.000000,1.000000,2.000000
